In [1]:
# This script is based on:
# https://www.tensorflow.org/get_started/mnist/pros

import sys
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.framework import graph_util as gu
from tensorflow.python.framework.graph_util import remove_training_nodes
from tensorflow.tools.graph_transforms import TransformGraph

# Import training data

In [2]:
mnist = input_data.read_data_sets("mnist_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting mnist_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting mnist_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

# Define Tensorflow Model

In [3]:
batch_size = 50

### Fully connected 2 layer NN

In [4]:
def deepnn(x):
  W_fc1 = weight_variable([784, 128], name='W_fc1')
  b_fc1 = bias_variable([128], name='b_fc1')
  a_fc1 = tf.add(tf.matmul(x, W_fc1), b_fc1, name="zscore")
  h_fc1 = tf.nn.relu(a_fc1)

  W_fc2 = weight_variable([128, 64], name='W_fc2')
  b_fc2 = bias_variable([64], name='b_fc2')
  a_fc2 = tf.add(tf.matmul(h_fc1, W_fc2), b_fc2, name="zscore")
  h_fc2 = tf.nn.relu(a_fc2)

  W_fc3 = weight_variable([64, 10], name='W_fc3')
  b_fc3 = bias_variable([10], name='b_fc3')
  logits = tf.add(tf.matmul(h_fc2, W_fc3), b_fc3, name="logits")
  y_pred = tf.argmax(logits, 1, name='y_pred')

  return y_pred, logits


def weight_variable(shape, name):
  """weight_variable generates a weight variable of a given shape."""
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial, name)


def bias_variable(shape, name):
  """bias_variable generates a bias variable of a given shape."""
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial, name)

### Specify inputs, outputs, and a cost function

In [5]:
# Reset default graph
tf.reset_default_graph()

# Create the model
x = tf.placeholder(tf.float32, [None, 784], name="x")

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 10], name="y")

# Build the graph for the deep net
y_pred, logits = deepnn(x)

with tf.name_scope("Loss"):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, 
                                                               logits=logits)
    loss = tf.reduce_mean(cross_entropy, name="cross_entropy_loss")
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss, name="train_step")
  
# Here we specify the output as "Prediction/y_pred", this will be important later
with tf.name_scope("Prediction"): 
    correct_prediction = tf.equal(y_pred, 
                                  tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")

# Train the model

In [6]:
sess = tf.Session()
# Initialize the variables (i.e. assign their default value)
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

for i in range(1, 20001):
  batch_images, batch_labels = mnist.train.next_batch(batch_size)
  feed_dict = {x: batch_images, y_: batch_labels}
  if i % 1000 == 0:
    train_accuracy = sess.run(accuracy, feed_dict=feed_dict)
    print('step %d, training accuracy %g' % (i, train_accuracy))
  sess.run(train_step, feed_dict=feed_dict)

step 1000, training accuracy 0.88
step 2000, training accuracy 0.94
step 3000, training accuracy 0.9
step 4000, training accuracy 0.92
step 5000, training accuracy 0.94
step 6000, training accuracy 1
step 7000, training accuracy 0.94
step 8000, training accuracy 0.92
step 9000, training accuracy 0.96
step 10000, training accuracy 0.96
step 11000, training accuracy 0.92
step 12000, training accuracy 0.94
step 13000, training accuracy 0.98
step 14000, training accuracy 0.98
step 15000, training accuracy 0.96
step 16000, training accuracy 1
step 17000, training accuracy 1
step 18000, training accuracy 0.98
step 19000, training accuracy 1
step 20000, training accuracy 1


## What is the final accuracy

In [7]:
print('test accuracy %g' % sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

test accuracy 0.9706


# Freeze the graph

In [8]:
saver.save(sess, "./chkps/mnist_model")
out_nodes = [y_pred.op.name]
print(out_nodes)

Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'W_fc1' has type str, but expected one of: int, long, bool
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'W_fc1' has type str, but expected one of: int, long, bool
[u'y_pred']


### Remove unnecessary training nodes

In [9]:
sub_graph_def = remove_training_nodes(sess.graph_def)

### Freeze Constants

In [10]:
sub_graph_def = gu.convert_variables_to_constants(sess, sub_graph_def, out_nodes)

INFO:tensorflow:Froze 6 variables.
INFO:tensorflow:Converted 6 variables to const ops.


### Save the graph to PB file

In [11]:
graph_path = tf.train.write_graph(sub_graph_def,
                                  "./models",
                                  "mnist_tf.pb",
                                  as_text=False)

print('written graph to: %s' % graph_path)

written graph to: ./mnist_model/deep_mlp.pb


In [12]:
# close session
sess.close()